# Constitution du dataset à partir des données exportées de CEGID expert

Dossiers exportés :
- Missions du cabinet sur 2017
- Facturation 2017
- Entrées et sorties des clients

## Importation des librairies

In [1]:
import pandas as pd
import numpy as np
import re
from scipy import stats

## Lecture de l'ensemble des missions du cabinet sur 2020
1 ligne = 1 mission réalisée par 1 assistant pour le compte d'un client

In [2]:
missions = pd.read_csv("./Data/missions_2017.txt", sep="\t", decimal='.')

In [3]:
missions.head(10)

,Code Mission,Mission,Exercice,Client,Raison sociale,DEPART CLIENT,Assistant,ASSOCIES/EC,Article,Libellé,Qté activité,Prix de vente,Total de vente,Département,SITE,Unnamed: 15
0,AJUR2017007869600,JUR,2017,06796,BV VAL DES MAUVES,NaN,335,DAC,069,DIVERS JURIDIQUE,"0,25","54,00","13,50",JUR,MEUNG SUR LOIRE,NaN
1,AJUR2017007869600,JUR,2017,06796,BV VAL DES MAUVES,NaN,335,DAC,060,SECRETARIAT JURIDIQUE ANNUEL,"0,50","56,00","28,00",JUR,MEUNG SUR LOIRE,NaN
2,ACOM2017008248000,COM,2017,05955,CROC EXPRESS,1.0,664,DAC,021,DECLARATIONS MENSUELLES / TRIMESTRIELLES,"0,50","72,00","36,00",SOCSJ,MEUNG SUR LOIRE,NaN
3,ACOM2017008333700,COM,2017,06834,ORLANDO GENNARO,NaN,271,FRL,020,FICHE DE PAIE,"0,50","56,00","28,00",SOCSJ,CLIENT PARTI,NaN
4,ACOM2017008310700,COM,2017,02395,BETMON,NaN,271,FRL,021,DECLARATIONS MENSUELLES / TRIMESTRIELLES,"0,50","56,00","28,00",SOCSJ,CLIENT PARTI,NaN
5,AJUR2017008549700,JUR,2017,07001,LOUIS DES P'TITS PLATS,4.0,666,VAL,060,SECRETARIAT JURIDIQUE ANNUEL,"0,25","56,00","14,00",JUR,GIEN,NaN
6,AJUR2017008201000,JUR,2017,77260,G.A.E.L.,NaN,340,JOP,060,SECRETARIAT JURIDIQUE ANNUEL,"1,25","66,00","82,50",JUR,CLIENT PARTI,NaN
7,AJUR2017008201000,JUR,2017,77260,G.A.E.L.,NaN,666,JOP,060,SECRETARIAT JURIDIQUE ANNUEL,"0,25","56,00","14,00",JUR,CLIENT PARTI,NaN
8,ACOM2017008303800,COM,2017,06671,ICOLE-DUMAS,NaN,271,SLE,020,FICHE DE PAIE,"1,25","56,00","70,00",SOCSJ,OLIVET,NaN
9,AJUR2017008201000,JUR,2017,77260,G.A.E.L.,NaN,340,JOP,069,DIVERS JURIDIQUE,"0,25","66,00","16,50",JUR,CLIENT PARTI,NaN


In [4]:
#Le dataset des missions 2017 contient 257 838 entrées et 16 colonnes
missions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257838 entries, 0 to 257837
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Code Mission     257838 non-null  object 
 1   Mission          257838 non-null  object 
 2   Exercice         257838 non-null  int64  
 3   Client           257838 non-null  object 
 4   Raison sociale   257838 non-null  object 
 5   DEPART CLIENT    62485 non-null   float64
 6   Assistant        257838 non-null  object 
 7   ASSOCIES/EC      257838 non-null  object 
 8   Article          257838 non-null  object 
 9   Libellé          257838 non-null  object 
 10  Qté activité     257838 non-null  object 
 11  Prix de vente    257838 non-null  object 
 12  Total  de vente  257838 non-null  object 
 13  Département      238346 non-null  object 
 14  SITE             257838 non-null  object 
 15  Unnamed: 15      0 non-null       float64
dtypes: float64(2), int64(1), object(13)
me

In [5]:
missions.columns

Index(['Code Mission', 'Mission', 'Exercice', 'Client', 'Raison sociale',
       'DEPART CLIENT', 'Assistant', 'ASSOCIES/EC', 'Article', 'Libellé',
       'Qté activité', 'Prix de vente', 'Total  de vente', 'Département',
       'SITE', 'Unnamed: 15'],
      dtype='object')

In [6]:
#Supprimer la dernière colonne inutile
missions.drop(columns=['Département','Unnamed: 15'], inplace=True)

In [7]:
missions.columns = ['code_mission','mission','exercice','code_client',
                    'client','depart_client','code_assistant','manager','code_article',
                    'libelle_article','temps','cout_horaire','valorisation_temps','site']

In [8]:
missions.dtypes

code_mission           object
mission                object
exercice                int64
code_client            object
client                 object
depart_client         float64
code_assistant         object
manager                object
code_article           object
libelle_article        object
temps                  object
cout_horaire           object
valorisation_temps     object
site                   object
dtype: object

In [9]:
missions.isna().sum()

code_mission               0
mission                    0
exercice                   0
code_client                0
client                     0
depart_client         195353
code_assistant             0
manager                    0
code_article               0
libelle_article            0
temps                      0
cout_horaire               0
valorisation_temps         0
site                       0
dtype: int64

## Modifier les colonnes catégorielles en float

In [10]:
#Remplacer les virgules par des points et espaces pour créer un nombre décimal
missions['temps'] = [x.replace(',', '.') for x in missions['temps']]
missions['temps'] = missions['temps'].astype(float)

In [11]:
missions['cout_horaire'] = [x.replace(',', '.') for x in missions['cout_horaire']]
missions['cout_horaire'] = [re.sub('\s','',x) for x in missions['cout_horaire']]
missions['cout_horaire'] = pd.to_numeric(missions['cout_horaire'])

In [12]:
missions['valorisation_temps'] = [x.replace(',', '.') for x in missions['valorisation_temps']]
missions['valorisation_temps'] = [re.sub('\s','',x) for x in missions['valorisation_temps']]
missions['valorisation_temps'] = pd.to_numeric(missions['valorisation_temps'])

In [13]:
missions

,code_mission,mission,exercice,code_client,client,depart_client,code_assistant,manager,code_article,libelle_article,temps,cout_horaire,valorisation_temps,site
0,AJUR2017007869600,JUR,2017,06796,BV VAL DES MAUVES,NaN,335,DAC,069,DIVERS JURIDIQUE,0.25,54.0,13.50,MEUNG SUR LOIRE
1,AJUR2017007869600,JUR,2017,06796,BV VAL DES MAUVES,NaN,335,DAC,060,SECRETARIAT JURIDIQUE ANNUEL,0.50,56.0,28.00,MEUNG SUR LOIRE
2,ACOM2017008248000,COM,2017,05955,CROC EXPRESS,1.0,664,DAC,021,DECLARATIONS MENSUELLES / TRIMESTRIELLES,0.50,72.0,36.00,MEUNG SUR LOIRE
3,ACOM2017008333700,COM,2017,06834,ORLANDO GENNARO,NaN,271,FRL,020,FICHE DE PAIE,0.50,56.0,28.00,CLIENT PARTI
4,ACOM2017008310700,COM,2017,02395,BETMON,NaN,271,FRL,021,DECLARATIONS MENSUELLES / TRIMESTRIELLES,0.50,56.0,28.00,CLIENT PARTI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257833,AGEJ2017010743500,GEJ,2017,06670,LA BUDANE,NaN,965,SLE,069,DIVERS JURIDIQUE,0.25,165.0,41.25,ORLEANS
257834,AGEJ2017010743500,GEJ,2017,06670,LA BUDANE,NaN,0R2,SLE,069,DIVERS JURIDIQUE,1.00,58.5,58.50,ORLEANS
257835,AGEJ2017010743500,GEJ,2017,06670,LA BUDANE,NaN,0R2,SLE,069,DIVERS JURIDIQUE,0.50,58.5,29.25,ORLEANS
257836,AJUR2017008930300,JUR,2017,06702,AVENIR R ET R,NaN,63P,SLE,060,SECRETARIAT JURIDIQUE ANNUEL,1.00,76.5,76.50,OLIVET


## Agrégation du dataframe sur le code_mission pour sommer le temps et la valorisation du temps

Il faut garder une version non aggrégée pour avoir :
- les différents codes assistants par manager
- Les différents sites
- Les différents code_article et libellé articles (activités réalisées)

In [15]:
missions.code_mission.values

array(['AJUR2017007869600', 'AJUR2017007869600', 'ACOM2017008248000', ...,
       'AGEJ2017010743500', 'AJUR2017008930300', 'AGEJ2017010743500'],
      dtype=object)

In [16]:
missions_agg = missions.groupby(['code_mission']).agg({'mission':[lambda x:x.value_counts().index[0]],
                                                        'exercice':[lambda x:x.value_counts().index[0]],
                                                        'code_client': [lambda x:x.value_counts().index[0]],
                                                        'code_assistant': ['nunique'],
                                                        'manager': [lambda x:x.value_counts().index[0]],
                                                        'code_article': [lambda x: list(x.unique())],
                                                        'libelle_article': [lambda x: list(x.unique())],
                                                        'temps': ['sum'],
                                                        'valorisation_temps': ['sum'],
                                                        'site': [lambda x:x.value_counts().index[0]]})

In [17]:
missions_agg.head()

,mission,exercice,code_client,code_assistant,manager,code_article,libelle_article,temps,valorisation_temps,site
,<lambda>,<lambda>,<lambda>,nunique,<lambda>,<lambda>,<lambda>,sum,sum,<lambda>
code_mission,,,,,,,,,,
AAGS2017010396900,AGS,2017,05782,2,JOP,[029],[AUTRES TRAVAUX EN SOCIAL],43.00,5231.25,PITHIVIERS
AAGS2017010397000,AGS,2017,06565,1,FRL,[001],[RENSEIGNEMENTS CLIENT],1.00,68.00,GIEN
AAGS2017010534300,AGS,2017,06924,2,JOP,"[029, 024, 001]","[AUTRES TRAVAUX EN SOCIAL, FORMALITES DE LICEN...",31.25,3868.00,PITHIVIERS
ACAB2017009616200,CAB,2017,09230,12,NON FACTURABLE,"[081, 096, 091, 079, 078, 082, 074, 088]","[COMPTABILITE GENERALE CABINET, DOCUMENTATION,...",2049.50,222874.75,NON FACTURABLE
ACAB2017009616300,CAB,2017,09000,98,NON FACTURABLE,"[083, 082, 079, 096, 075, 010, 092, 080, 081, ...","[NON FACTURABLE SOCIAL, NON FACTURABLE COMPTAB...",2279.64,151143.13,NON FACTURABLE


In [18]:
missions_agg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7921 entries, AAGS2017010396900 to ASST2017010135000
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   (mission, <lambda>)          7921 non-null   object 
 1   (exercice, <lambda>)         7921 non-null   int64  
 2   (code_client, <lambda>)      7921 non-null   object 
 3   (code_assistant, nunique)    7921 non-null   int64  
 4   (manager, <lambda>)          7921 non-null   object 
 5   (code_article, <lambda>)     7921 non-null   object 
 6   (libelle_article, <lambda>)  7921 non-null   object 
 7   (temps, sum)                 7921 non-null   float64
 8   (valorisation_temps, sum)    7921 non-null   float64
 9   (site, <lambda>)             7921 non-null   object 
dtypes: float64(2), int64(2), object(6)
memory usage: 680.7+ KB


## Lecture du dossier facturation sur 2017
Quel coût facturé au client ?

In [19]:
facturation = pd.read_csv("./Data/facturation_2017.txt", sep="\t")

In [20]:
facturation.head()

,Code mission,Client,Exercice,Mission,Total HT,Exercice.1,Date,Nature,N° Pièce,Article,Qté facturée,Prix unitaire,Numéro d'ordre,Désignation,Raison sociale,Unnamed: 15
0,AJUR2017008787500,05955,2017,JUR,"97,50",2017,23/12/2015,FAC,647073,601,0,"390,00",5,SECRETARIAT JURIDIQUE ACOMPTE N°1,CROC EXPRESS,NaN
1,AGEJ2017008907500,07653,2017,GEJ,"830,00",2017,30/03/2016,FAC,650700,061,1,"830,00",1,FORMALITES DE CONSTITUTION DE LA,TAXI FLORAN,NaN
2,ACOM2017008058400,03851,2017,COM,"1 210,00",2017,15/04/2016,FAC,650809,ACOMPTE,1,"1 210,00",1,ACOMPTE PROVISIONNEL N° 1,VANDEKERKHOVE,NaN
3,ACOM2017008058800,05949,2017,COM,"710,00",2017,15/04/2016,FAC,650814,ACOMPTE,1,"710,00",1,ACOMPTE PROVISIONNEL N° 1,HENRY,NaN
4,ACOM2017008058900,06043,2017,COM,"505,00",2017,15/04/2016,FAC,650819,ACOMPTE,1,"505,00",1,ACOMPTE PROVISIONNEL N° 1,C3H,NaN


In [21]:
facturation.dtypes

Code mission       object
Client             object
Exercice            int64
Mission            object
Total HT           object
Exercice.1          int64
Date               object
Nature             object
N° Pièce            int64
Article            object
Qté facturée        int64
Prix unitaire      object
Numéro d'ordre      int64
Désignation        object
Raison sociale     object
Unnamed: 15       float64
dtype: object

In [22]:
facturation[facturation['Exercice'].isna()]

,Code mission,Client,Exercice,Mission,Total HT,Exercice.1,Date,Nature,N° Pièce,Article,Qté facturée,Prix unitaire,Numéro d'ordre,Désignation,Raison sociale,Unnamed: 15


In [23]:
facturation.isnull().sum()

Code mission          0
Client                0
Exercice              0
Mission               0
Total HT              0
Exercice.1            0
Date                  0
Nature                0
N° Pièce              0
Article               0
Qté facturée          0
Prix unitaire         0
Numéro d'ordre        0
Désignation          22
Raison sociale        0
Unnamed: 15       36886
dtype: int64

In [24]:
# Remplacer les valeurs catégorielles par des floats
facturation['Total HT'] = [str(x).replace(',', '.') for x in facturation['Total HT']]
facturation['Total HT'] = [re.sub('\s','',x) for x in facturation['Total HT']]
facturation['Total HT'] = pd.to_numeric(facturation['Total HT'])

In [25]:
facturation['Total HT'].isnull().sum()

0

### Remplir les valeurs NaN par 0 dans le Total HT de la facturation 

In [26]:
facturation.fillna(0, inplace=True)

In [27]:
facturation['Qté facturée'].value_counts().head(10)

1    25150
2     3870
3     1818
4     1283
5      932
6      717
7      475
8      338
0      331
9      286
Name: Qté facturée, dtype: int64

## Agrégation du dataset facturation sur le code_mission pour sommer le total HT facturé au client sur une mission particulière

In [28]:
fact_agg = facturation.groupby(['Code mission']).agg({'Total HT':['sum']})

In [29]:
fact_agg.head()

,Total HT
,sum
Code mission,
AAGS2017010396900,4550.00
AAGS2017010397000,750.00
AAGS2017010534300,4804.20
AANT2017009877900,2953.26
AANT2017010071600,1390.73


Il existe plus de missions dans facturation que de missions dans le dataset mission ?

## Concaténation du dataset facturation sur l'ensemble des missions

In [30]:
data = pd.concat([missions_agg, fact_agg], axis=1)

In [31]:
data.columns = ['mission','exercice','code_client','code_assistant','manager','code_article',
                'libelle_article','temps','valorisation_temps','site','facturation']

In [32]:
#294 factures à 0 euro
data.isna().sum()

mission               154
exercice              154
code_client           154
code_assistant        154
manager               154
code_article          154
libelle_article       154
temps                 154
valorisation_temps    154
site                  154
facturation           352
dtype: int64

In [33]:
len(data.index)

8075

In [34]:
#Remplacer NaN par 0 euro
data['facturation'].fillna(0, inplace=True)

In [35]:
#Le reste des valeurs nulles correspondent au facture sur des codes_missions qui n'existent pas dans le dataset missions
data.isna().sum()

mission               154
exercice              154
code_client           154
code_assistant        154
manager               154
code_article          154
libelle_article       154
temps                 154
valorisation_temps    154
site                  154
facturation             0
dtype: int64

Il se peut que des missions ne soient pas terminées sur le millésime 2020 mais qu'il existe une facture

Supprimer les 154 lignes ajoutées sans aucune donnée car une facture mais pas dans de code_mission dans le dataset missions

In [36]:
list_index = data[data['code_client'].isna()].index
len(list_index)

154

In [37]:
list_index

Index(['AANT2017009877900', 'AANT2017010071600', 'AANT2017010082500',
       'AANT2017010121400', 'AANT2017010232600', 'AANT2017010233200',
       'AANT2017010435400', 'AANT2017010441700', 'AANT2017010453600',
       'AANT2017010461200',
       ...
       'AGES2017011439600', 'AJUR2017008778800', 'AJUR2017009607800',
       'AJUR2017009694300', 'AJUR2017009732700', 'AJUR2017009732800',
       'AJUR2017009746400', 'AJUR2017012502600', 'ALOC2017010095900',
       'ALOC2017010441500'],
      dtype='object', length=154)

In [38]:
facturation.columns

Index(['Code mission', 'Client', 'Exercice', 'Mission', 'Total HT',
       'Exercice.1', 'Date', 'Nature', 'N° Pièce', 'Article', 'Qté facturée',
       'Prix unitaire', 'Numéro d'ordre', 'Désignation', 'Raison sociale',
       'Unnamed: 15'],
      dtype='object')

In [39]:
facturation[facturation['Code mission'] == 'AANT2017009877900']

,Code mission,Client,Exercice,Mission,Total HT,Exercice.1,Date,Nature,N° Pièce,Article,Qté facturée,Prix unitaire,Numéro d'ordre,Désignation,Raison sociale,Unnamed: 15
697,AANT2017009877900,06791,2017,ANT,2953.26,2017,16/11/2016,FAC,659958,001,1,"2 953,26",1,REGULARISATION ANTERIORITE,LES TRESORS DE NEPTUNE,0.0


In [40]:
data.drop(list_index, axis=0, inplace=True)

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7921 entries, AAGS2017010396900 to ASST2017010135000
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   mission             7921 non-null   object 
 1   exercice            7921 non-null   float64
 2   code_client         7921 non-null   object 
 3   code_assistant      7921 non-null   float64
 4   manager             7921 non-null   object 
 5   code_article        7921 non-null   object 
 6   libelle_article     7921 non-null   object 
 7   temps               7921 non-null   float64
 8   valorisation_temps  7921 non-null   float64
 9   site                7921 non-null   object 
 10  facturation         7921 non-null   float64
dtypes: float64(5), object(6)
memory usage: 742.6+ KB


In [42]:
data.head()

,mission,exercice,code_client,code_assistant,manager,code_article,libelle_article,temps,valorisation_temps,site,facturation
AAGS2017010396900,AGS,2017.0,05782,2.0,JOP,[029],[AUTRES TRAVAUX EN SOCIAL],43.00,5231.25,PITHIVIERS,4550.0
AAGS2017010397000,AGS,2017.0,06565,1.0,FRL,[001],[RENSEIGNEMENTS CLIENT],1.00,68.00,GIEN,750.0
AAGS2017010534300,AGS,2017.0,06924,2.0,JOP,"[029, 024, 001]","[AUTRES TRAVAUX EN SOCIAL, FORMALITES DE LICEN...",31.25,3868.00,PITHIVIERS,4804.2
ACAB2017009616200,CAB,2017.0,09230,12.0,NON FACTURABLE,"[081, 096, 091, 079, 078, 082, 074, 088]","[COMPTABILITE GENERALE CABINET, DOCUMENTATION,...",2049.50,222874.75,NON FACTURABLE,0.0
ACAB2017009616300,CAB,2017.0,09000,98.0,NON FACTURABLE,"[083, 082, 079, 096, 075, 010, 092, 080, 081, ...","[NON FACTURABLE SOCIAL, NON FACTURABLE COMPTAB...",2279.64,151143.13,NON FACTURABLE,0.0


## Ajouter les entrées et sorties des clients MCCM

In [43]:
clients = pd.read_csv("./Data/client_dates.csv", sep=";")

In [44]:
clients_formes = pd.read_csv("./Data/client-naf-forme.csv", sep=";")

In [45]:
clients.head()

,Code,Date création,Départ cabinet
0,30,07/11/1995,0
1,44,31/10/1995,0
2,441,30/10/1995,0
3,455,07/11/1995,0
4,549,09/11/1995,0


In [46]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11281 entries, 0 to 11280
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Code            11281 non-null  object
 1   Date création   11277 non-null  object
 2   Départ cabinet  11281 non-null  object
dtypes: object(3)
memory usage: 264.5+ KB


In [47]:
clients.dtypes

Code              object
Date création     object
Départ cabinet    object
dtype: object

In [48]:
def entree_sortie(col):
    
    for date in str(col):
        if '2017' in str(col):
            return 1
        else:
            return 0

In [49]:
#Test de la fonction
entree_sortie("20/02/2017")

1

In [50]:
clients['entrée_clt'] = clients['Date création'].apply(entree_sortie)

In [51]:
clients['sortie_clt'] = clients['Départ cabinet'].apply(entree_sortie)

In [52]:
clients.head(50)

,Code,Date création,Départ cabinet,entrée_clt,sortie_clt
0,30,07/11/1995,0,0,0
1,44,31/10/1995,0,0,0
2,441,30/10/1995,0,0,0
3,455,07/11/1995,0,0,0
4,549,09/11/1995,0,0,0
5,569,06/11/1995,0,0,0
6,579,06/11/1995,31/03/2020,0,0
7,584,10/11/1995,0,0,0
8,613,16/11/1995,0,0,0
9,632,30/10/1995,0,0,0


In [53]:
clients.columns

Index(['Code', 'Date création', 'Départ cabinet', 'entrée_clt', 'sortie_clt'], dtype='object')

In [54]:
clients.columns = ['code_client', 'ENTREE_CLIENT', 'SORTIE_CLIENT', 'entrée_clt', 'sortie_clt']

In [55]:
clients.code_client.value_counts()

2498      1
8936      1
6008      1
A1619     1
TESTA1    1
         ..
6715      1
7743      1
6426      1
2810      1
2241      1
Name: code_client, Length: 11281, dtype: int64

In [56]:
clients_formes.head()

,Code,Code NAF,Forme juridique
0,9,Divers,SA
1,25,Autre imprimerie (labeur),SARL
2,26,Édition de revues et périodiques,SA
3,30,Fabrication de parquets assemblés,SAS
4,36,Commerce de détail d'habillement en magasin sp...,SARL


## Concaténer les entrées et les sorties au dataset selon le code_client avec la fonction merge()

In [57]:
clients.code_client

0           30
1           44
2          441
3          455
4          549
         ...  
11276    P7129
11277    P7503
11278    P7540
11279    P7673
11280    P7911
Name: code_client, Length: 11281, dtype: object

In [58]:
len(clients.code_client[0])

2

In [59]:
def suppr_0(s):
    return s.lstrip("0")

In [60]:
#Test de la suppression de 0 au début des identifiants
suppr_0("06565")

'6565'

In [61]:
data.code_client = data.code_client.apply(suppr_0)

In [62]:
data.head()

,mission,exercice,code_client,code_assistant,manager,code_article,libelle_article,temps,valorisation_temps,site,facturation
AAGS2017010396900,AGS,2017.0,5782,2.0,JOP,[029],[AUTRES TRAVAUX EN SOCIAL],43.00,5231.25,PITHIVIERS,4550.0
AAGS2017010397000,AGS,2017.0,6565,1.0,FRL,[001],[RENSEIGNEMENTS CLIENT],1.00,68.00,GIEN,750.0
AAGS2017010534300,AGS,2017.0,6924,2.0,JOP,"[029, 024, 001]","[AUTRES TRAVAUX EN SOCIAL, FORMALITES DE LICEN...",31.25,3868.00,PITHIVIERS,4804.2
ACAB2017009616200,CAB,2017.0,9230,12.0,NON FACTURABLE,"[081, 096, 091, 079, 078, 082, 074, 088]","[COMPTABILITE GENERALE CABINET, DOCUMENTATION,...",2049.50,222874.75,NON FACTURABLE,0.0
ACAB2017009616300,CAB,2017.0,9000,98.0,NON FACTURABLE,"[083, 082, 079, 096, 075, 010, 092, 080, 081, ...","[NON FACTURABLE SOCIAL, NON FACTURABLE COMPTAB...",2279.64,151143.13,NON FACTURABLE,0.0


In [63]:
data[data['code_client'] == '6087SAS'].index

Index(['ACOM2017009221900', 'ASOP2017010281600'], dtype='object')

In [64]:
data['code_client'].replace(['6087SAS'], '06087SAS', inplace=True)

In [65]:
data.reset_index(inplace=True)

In [66]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7921 entries, 0 to 7920
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               7921 non-null   object 
 1   mission             7921 non-null   object 
 2   exercice            7921 non-null   float64
 3   code_client         7921 non-null   object 
 4   code_assistant      7921 non-null   float64
 5   manager             7921 non-null   object 
 6   code_article        7921 non-null   object 
 7   libelle_article     7921 non-null   object 
 8   temps               7921 non-null   float64
 9   valorisation_temps  7921 non-null   float64
 10  site                7921 non-null   object 
 11  facturation         7921 non-null   float64
dtypes: float64(5), object(7)
memory usage: 742.7+ KB


In [67]:
data_2017 = pd.merge(data, clients[['code_client','entrée_clt', 'sortie_clt']], how='left', on="code_client")

In [68]:
clients_formes.head()

,Code,Code NAF,Forme juridique
0,9,Divers,SA
1,25,Autre imprimerie (labeur),SARL
2,26,Édition de revues et périodiques,SA
3,30,Fabrication de parquets assemblés,SAS
4,36,Commerce de détail d'habillement en magasin sp...,SARL


In [69]:
clients_formes.columns

Index(['Code', 'Code NAF', 'Forme juridique'], dtype='object')

In [70]:
clients_formes.columns = ['code_client', 'Code NAF', 'Forme juridique']

In [71]:
data_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7921 entries, 0 to 7920
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               7921 non-null   object 
 1   mission             7921 non-null   object 
 2   exercice            7921 non-null   float64
 3   code_client         7921 non-null   object 
 4   code_assistant      7921 non-null   float64
 5   manager             7921 non-null   object 
 6   code_article        7921 non-null   object 
 7   libelle_article     7921 non-null   object 
 8   temps               7921 non-null   float64
 9   valorisation_temps  7921 non-null   float64
 10  site                7921 non-null   object 
 11  facturation         7921 non-null   float64
 12  entrée_clt          7921 non-null   int64  
 13  sortie_clt          7921 non-null   int64  
dtypes: float64(5), int64(2), object(7)
memory usage: 928.2+ KB


In [72]:
data_final_2017 = pd.merge(data_2017, clients_formes[['code_client', 'Code NAF','Forme juridique']], how='left', on="code_client")

In [74]:
data_final_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7921 entries, 0 to 7920
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               7921 non-null   object 
 1   mission             7921 non-null   object 
 2   exercice            7921 non-null   float64
 3   code_client         7921 non-null   object 
 4   code_assistant      7921 non-null   float64
 5   manager             7921 non-null   object 
 6   code_article        7921 non-null   object 
 7   libelle_article     7921 non-null   object 
 8   temps               7921 non-null   float64
 9   valorisation_temps  7921 non-null   float64
 10  site                7921 non-null   object 
 11  facturation         7921 non-null   float64
 12  entrée_clt          7921 non-null   int64  
 13  sortie_clt          7921 non-null   int64  
 14  Code NAF            7921 non-null   object 
 15  Forme juridique     7921 non-null   object 
dtypes: flo

In [75]:
#Vérification des valeurs nulles
data_final_2017.isna().sum()

index                 0
mission               0
exercice              0
code_client           0
code_assistant        0
manager               0
code_article          0
libelle_article       0
temps                 0
valorisation_temps    0
site                  0
facturation           0
entrée_clt            0
sortie_clt            0
Code NAF              0
Forme juridique       0
dtype: int64

In [77]:
#Vérification de combien de lignes par client
data_final_2017.code_client.value_counts()

7406     7
7101     7
A0554    6
7965     6
3262     6
        ..
5012     1
7224     1
6833     1
A0241    1
A0465    1
Name: code_client, Length: 4000, dtype: int64

In [78]:
data_final_2017[data_final_2017['code_client']=='5782']

,index,mission,exercice,code_client,code_assistant,manager,code_article,libelle_article,temps,valorisation_temps,site,facturation,entrée_clt,sortie_clt,Code NAF,Forme juridique
0,AAGS2017010396900,AGS,2017.0,5782,2.0,JOP,[029],[AUTRES TRAVAUX EN SOCIAL],43.00,5231.25,PITHIVIERS,4550.0,0,1,Fabrication de charpentes et d'autres menuiseries,SAS
2091,ACOM2017009212800,COM,2017.0,5782,5.0,JOP,"[049, 043, 016, 001, 011, 018, 041, 046, 044, ...","[AUTRES TRAVAUX EXCEPTIONNELS, TABLEAU DE BORD...",173.50,22158.25,PITHIVIERS,24500.0,0,1,Fabrication de charpentes et d'autres menuiseries,SAS
3786,AGE02017010748500,GE0,2017.0,5782,3.0,JOP,"[049, 019, 029, 044]","[AUTRES TRAVAUX EXCEPTIONNELS, PRESENTATION DE...",89.25,13409.25,PITHIVIERS,13955.0,0,1,Fabrication de charpentes et d'autres menuiseries,SAS
4568,AGES2017010232900,GES,2017.0,5782,4.0,JOP,"[025, 027, 029, 024]","[CONTRATS DE TRAVAIL, REPRESENTATION DU PERSON...",36.25,2708.75,PITHIVIERS,2045.0,0,1,Fabrication de charpentes et d'autres menuiseries,SAS
6787,ASOP2017010235000,SOP,2017.0,5782,11.0,JOP,"[029, 021, 030, 022, 020, F0211, F02001, F0200...","[AUTRES TRAVAUX EN SOCIAL, DECLARATIONS MENSUE...",1011.25,27418.35,PITHIVIERS,17419.0,0,1,Fabrication de charpentes et d'autres menuiseries,SAS


In [79]:
data_final_2017.dtypes

index                  object
mission                object
exercice              float64
code_client            object
code_assistant        float64
manager                object
code_article           object
libelle_article        object
temps                 float64
valorisation_temps    float64
site                   object
facturation           float64
entrée_clt              int64
sortie_clt              int64
Code NAF               object
Forme juridique        object
dtype: object

In [ ]:
#Supprimer les dossiers internes qui sont pour le cabinet et non facturable

In [85]:
data_final_2017[data_final_2017['type_mission'] == "SST"].index

Int64Index([7919, 7920], dtype='int64')

In [86]:
data_final_2017.drop([7919, 7920], axis=0, inplace=True)

## Exportation du dataset 2017

In [87]:
data_final_2017.columns = ['code_mission', 'type_mission', 'exercice', 'code_client', 'total_assistant',
                           'manager', 'code_article', 'libelle_article', 'temps',
                           'valorisation_temps', 'site', 'facturation', 'entrée_clt', 'sortie_clt', 'secteur', 'forme']

In [88]:
data_final_2017.head()

,code_mission,type_mission,exercice,code_client,total_assistant,manager,code_article,libelle_article,temps,valorisation_temps,site,facturation,entrée_clt,sortie_clt,secteur,forme
0,AAGS2017010396900,AGS,2017.0,5782,2.0,JOP,[029],[AUTRES TRAVAUX EN SOCIAL],43.00,5231.25,PITHIVIERS,4550.0,0,1,Fabrication de charpentes et d'autres menuiseries,SAS
1,AAGS2017010397000,AGS,2017.0,6565,1.0,FRL,[001],[RENSEIGNEMENTS CLIENT],1.00,68.00,GIEN,750.0,0,0,Activités d'architecture,EURL
2,AAGS2017010534300,AGS,2017.0,6924,2.0,JOP,"[029, 024, 001]","[AUTRES TRAVAUX EN SOCIAL, FORMALITES DE LICEN...",31.25,3868.00,PITHIVIERS,4804.2,0,0,Fabrication de meubles de bureau et de magasin,SAS
3,ACAB2017009616200,CAB,2017.0,9230,12.0,NON FACTURABLE,"[081, 096, 091, 079, 078, 082, 074, 088]","[COMPTABILITE GENERALE CABINET, DOCUMENTATION,...",2049.50,222874.75,NON FACTURABLE,0.0,0,0,Divers,Particulier
4,ACAB2017009616300,CAB,2017.0,9000,98.0,NON FACTURABLE,"[083, 082, 079, 096, 075, 010, 092, 080, 081, ...","[NON FACTURABLE SOCIAL, NON FACTURABLE COMPTAB...",2279.64,151143.13,NON FACTURABLE,0.0,0,0,Divers,Particulier


In [90]:
data_final_2017.to_csv("./Data/dataset-2017.csv", index=False)